In [ ]:
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats

import sys

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/11-Python_Functions/")

In [ ]:
from Functions import is_outlier

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal", "ccbar","All"]
# samples = ["Signal", "ccbar","charged","ddbar","mixed","ssbar","taupair","uubar"]

In [ ]:
DataFrames = {}  # define empty dictionary to hold dataframes
Date = "1227"
Attempt = "0"

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:gammaV0tree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"24_"+ Attempt +"_"+ s +".root:gammaV0tree",library='pd')

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

In [ ]:
DataFrames["All"].isna().sum()

The line below is to look at the available variables.

In [ ]:
DataFrames["Signal"].columns.tolist()

The line below only looks at the interesting variables.

In [ ]:
features = DataFrames["All"].columns.tolist()

truth = []

In [ ]:
# # Suffix removal from String list
# # using list comprehension + endswith()
# features = [ele for ele in features if not ele.endswith('isCloneTrack')
#             and not ele.endswith('charge')
#             and not ele.endswith('mcPDG')
#             and not ele.endswith('mcErrors')
#             and not ele.endswith('genMotherPDG')
#             and not ele.endswith('genMotherPDG_1')
#             and not ele.endswith('genMotherPDG_2')
#             and not ele.endswith('genMotherID')
#             and not ele.endswith('genMotherID_1')
#             and not ele.endswith('genMotherID_2')
#             and not ele.endswith('genParticleID')
#             and not ele.endswith('nMCDaughters') 
#             and not ele.endswith('mcInitial')
#             and not ele.endswith('mcPrimary')
#             and not ele.endswith('mcPhotos')
#             and not ele.endswith('isMisidentified')
#             and not ele.endswith('InvM')]

In [ ]:
# Initializing remove list
remove_list = ['__experiment__', '__run__', '__event__', '__production__', '__candidate__', '__ncandidates__', '__weight__']
 
# Removing elements present in other list
# using list comprehension
features = [i for i in features if i not in remove_list]

In [ ]:
features

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# # Electron ID
# #-------------------
# # DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# # DataFrames["All"] = DataFrames["All"][DataFrames["All"]['e_electronID']>=0.95]
# # DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# # DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_em_electronID']>=0.95]

# # Photon Conversion
# #-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# # Peaking Background Removal
# #----------------------------
# # DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_diff_D0pi']>=0.15)]
# # DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # # Vertex Fitting
# # #----------------
# # DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# # DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb']>=0.01]

# # Dalitz Removal
# #----------------------------
# # DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["All"]['Ds_pi0veto_M_Correction']>=0.16)]
# # DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# # Vertex Fit
# #----------------
# # DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb_Ds_rank']==1]
# # DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb_Ds_rank']==1]

# # D0 Invariant Mass
# #-----------------------
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_D0_sideband']==1)]
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_D0_sideband']==1)]

# # KaonID
# #-----------------
# # DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['K_kaonID']>=0.5]
# # DataFrames["All"] = DataFrames["All"][DataFrames["All"]['K_kaonID']>=0.5]

### Variable Comparison
Below, the for loop attempts to go through the different variable to see which holds the most discriminating power.

In [ ]:
Stacked = False
Density = True
Bins = 50
Signal = True

Op = 0.58
dM = 0.1
Hits = -1

# label1= 'failed'
# label2= r'$e^{+}\;\mathrm{from}\;\gamma$'
# label3= r'$e^{+}\;\mathrm{from}\;\pi^{0}$'
# label4= 'misID'

for i in features:
    label3= r'$Background$'
    #             label3= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'

    labels=[label3]
    colors=['C4']
    data=[
        DataFrames["All"][i],
        DataFrames["Signal"][i]
    ]
    #             data=[DataFrames["All"][i]]

    #             z_scores_0 = stats.zscore(data[0])
    #             z_scores_1 = stats.zscore(data[1])
    #             data[0] = data[0][(z_scores_0 > -3) & (z_scores_0 < 3)]
    #             data[1] = data[1][(z_scores_1 > -3) & (z_scores_1 < 3)]
    #             Range = [0,0.5]
    #             Range = [-0.1,0.1]
    Range = [data[1].min(axis=0),data[1].max(axis=0)]

    plt.hist(data[1], label=r'$Signal$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
    plt.hist(data[0], color=colors[0], label=labels[0], density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

    plt.ylabel('Entries')
    plt.xlabel(i)
    #             plt.yscale("log")
    plt.legend()
    plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
Signal = True

Op = 0.58
dM = 0.1
Hits = -1

# label1= 'failed'
# label2= r'$e^{+}\;\mathrm{from}\;\gamma$'
# label3= r'$e^{+}\;\mathrm{from}\;\pi^{0}$'
# label4= 'misID'

for i in features:

    label1= r'$Hits = 0$'
    label2= r'$Hits > 0$'

    labels=[label1,label2]
    colors=['#fd7f6f','#7eb0d5']

    data = [DataFrames[samples[0]][(DataFrames[samples[0]]['Ds_isSignal']==1) & (DataFrames["Signal"]['e_firstPXDLayer']==-1)][i],
            DataFrames[samples[0]][(DataFrames[samples[0]]['Ds_isSignal']==1) & (DataFrames["Signal"]['e_firstPXDLayer']>-1)][i],
           ]
    
    Range = [-0.05,0.05]
#     Range = [-1,1]
#     Range = [0,1]
#     Range = [DataFrames['Signal'][i].min(axis=0),DataFrames['Signal'][i].max(axis=0)]

    plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')

    # Title
    #---------
    # Signal
    plt.title(r'$isSignal(D_s^{+})=1$', loc = "left")
    plt.title(r'$\bf Signal\;Events$', loc = "right")
    # # Background
    # plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
    # plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
    # Label
    #---------
    # plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
    plt.ylabel('Entries')
    # plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
    plt.xlabel(i)
    # plt.yscale("log") 
    plt.legend()
    plt.show()

In [ ]:
print(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)]['e_mcSecPhysProc'].value_counts(dropna=False))
print(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)]['e_genMotherPDG'].value_counts(dropna=False))
print(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)]['Ds_genNStepsToDaughter_1'].value_counts(dropna=False))

In [ ]:
print(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423) & (abs(DataFrames["ccbar"]['e_genMotherPDG'])==111)]['e_firstPXDLayer'].value_counts(dropna=False))

### Single Variable  
The code below is to look at any specific variable closely.

Background Not Catergorized

In [ ]:
features

In [ ]:
Density = True
Bins = 50
i = 'Ds_gammaROE_InvM'
# i = 'Ds_gammaV0_mcDecayVertexFromIPDistance'
# i = 'Ds_MminusMtrue_D0e_pimass'
Range = [0, 0.4]
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

plt.hist(DataFrames["Signal"][i], label="Signal", density=Density, bins=Bins, alpha=0.8, range=Range)
plt.hist(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)][i], label="Background", density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

# Label
#--------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m\;[GeV/c^{2}]$')
# plt.yscale("log") 
plt.legend()
plt.show()

In [ ]:
Density = True
Bins = 50
i = 'Ds_mcM_D0e_emass'
# i = 'Ds_MminusMtrue_D0e_pimass'
Range = [1.8, 2.2]
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

plt.hist(DataFrames["Signal"][i], label="Signal", density=Density, bins=Bins, alpha=0.8, range=Range)
plt.hist(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)][i], label="Background", density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

# Label
#--------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m\;[GeV/c^{2}]$')
# plt.yscale("log") 
plt.legend()
plt.show()

In [ ]:
DataFrames["ccbar"].isna().sum()

In [ ]:
GoodConversionVariablesVeto = ['e_d0',
                               'e_dcosTheta'
                               'e_abs_dr',
                               'Ds_useRestFrame_daughterDiffOf_0_1_p',
                               'Ds_useRestFrame_daughterAngle_0_1',
                               'Ds_azimuthalAngleInDecayPlane_0_1'
                               'Ds_useRestFrame_daughterMotherDiffOf_0_p', # There are two unique areas
                               'Ds_flightDistance' # The signal is slightly shifted upwards
                               'Ds_daughterDiffOf_0_1_x', #Slight Shift
                               'Ds_daughterDiffOf_0_1_y', #Slight Shift
                               'Ds_daughterDiffOf_0_1_distance', #Slight Shift
                               'Ds_x', # Clear Shift
                               'Ds_y', # Clear Shift
                              ]

# Signal Electron vs ROE Electron Comparison

In [ ]:
Stacked = False
Density = False
Bins = 50
Range = [0, 160]
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

plt.hist(DataFrames[samples[1]][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)]['e_trackLength'], label=r'$e_{sig}^{+}$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.hist(DataFrames[samples[1]][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)]['Ds_gammaveto_em_trackLength'], label=r'$e_{ROE}^{-}$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

# Title
#---------
# Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# plt.title(r'$2M \; Events$', loc = "left")
# plt.title(r'$\bf Signal\;Events$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries$')
plt.xlabel(r'$trackLength \; [cm]$')
# plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
Range = [-3, 3]
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

plt.hist(DataFrames[samples[1]][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)]['e_d0'], label=r'$e_{sig}^{+}$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.hist(DataFrames[samples[1]][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)]['Ds_gammaveto_em_d0'], label=r'$e_{ROE}^{-}$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

# Title
#---------
# Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# plt.title(r'$2M \; Events$', loc = "left")
# plt.title(r'$\bf Signal\;Events$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries$')
plt.xlabel(r'$d_{0} \; [cm]$')
# plt.yscale("log")
plt.legend()
plt.show()

### 2-D Histogram

In [ ]:
# # name_x = 'Ds_massDifference_0'
# name_x = 'Ds_dM_D0pi'
# name_y = 'Ds_Angle_D0e'
# Sample = "ccbar"

# h = make_2d_hist([DataFrames[Sample][name_x], DataFrames[Sample][name_y]], bins=[50, 50], range=[[0.135,0.4],[0,3.14]])

# (
#     fig,
#     ax_2d,
#     ax_x_projection,
#     ax_y_projection,
#     ax_colorbar,
# ) = plot_2d_hist_with_projections(
#     h,
#     xlabel=r'$\Delta m\;[GeV/c^{2}]$',
#     ylabel=r'$\theta(D^{0}e^{+})\;[rad]$',
#     ylabel_x_projection="Entries",
#     xlabel_y_projection="Entries",
#     offset_x_labels=True,
#     colorbar_kwargs={"label": "Entries"},
#     figsize=(6,6)
# )

In [ ]:
Stacked = False
Density = True
Bins = 50
Signal = True
i = "Ds_chiProb"

# label1= 'failed'
# label2= r'$e^{+}\;\mathrm{from}\;\gamma$'
# label3= r'$e^{+}\;\mathrm{from}\;\pi^{0}$'
# label4= 'misID'

label3= r'$D^{*0}$'

labels=[label3]
colors=['C3']
data=[DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423)][i]]

#             Range = [0,0.5]
#             Range = [-0.2,0.2]
Range = [DataFrames["ccbar"][i].min(axis=0),DataFrames["ccbar"][i].max(axis=0)]

plt.hist(DataFrames["Signal"][i], color='C6', label="Signal", density=Density, bins=Bins, alpha=1, range=Range)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

plt.ylabel('Entries')
plt.xlabel(r'$\chi^2 \; Probability (D_s^{+})$')
plt.legend()
plt.show()